# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [7]:
import numpy as np


class Znw():
    q=17
    f=np.array([1,0,0,0,1])
    def __init__(self,vec):
        _,r=np.polydiv(np.array(vec),self.f)
        self.vec=r%17

    def __repr__(self):
        return str(self.vec)

    def __add__(self,other):
        if isinstance(other,Znw):
            return Znw(np.polyadd(self.vec,other.vec))
        else:
            raise TypeError(f"unsupported operand type(s) for +: 'Znw' and '{type(other).__name__}'")

    def __sub__(self,other):
        if isinstance(other,Znw):
            return Znw(np.polyadd(self.vec,-1*other.vec))
        else:
            raise TypeError(f"unsupported operand type(s) for +: 'Znw' and '{type(other).__name__}'")


    def __mul__(self,other):
        if isinstance(other,Znw):
            return Znw(np.polymul(self.vec,other.vec))
        elif isinstance(other,int):
            return Znw(other*self.vec)
        else:
            raise TypeError(f"unsupported operand type(s) for *: 'Znw' and '{type(other).__name__}'")

    __rmul__=__mul__


## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

In [8]:
K = 2
q = 17

#pierścień Z17[X]/X^4 + 1

prop = 0.1

n = 4

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [9]:
A = np.array([
  [Znw([6, 16, 16, 11]), Znw([9, 4, 6, 3 ])],
  [Znw([5, 3,  10, 1 ]), Znw([6, 1, 9, 15])],
])
s = np.array(
  [Znw([-1, -1, 1, 0]), Znw([-1, 0, -1, 0])]
)
e = np.array(
  [Znw([1, 0, 0]), Znw([1, -1, 0])]
)

t = A @ s + e
print("test: ", t, end="\n\n")

def key_gen():
    def generate_A():
        A = np.empty((K, K), dtype=object)
        for i in range(K):
            for j in range(K):
                new_array = np.random.randint(0, q + 1, size=n)
                A[i, j] = Znw(new_array)
        return A

    def generate_s():
        s = np.empty(K, dtype=object)
        for i in range(K):
            new_array = np.array([B_ni_1(prop) for _ in range(n)])
            s[i] = Znw(new_array)
        return s

    def generate_e():
        return generate_s()

    def calculate_t(A, s, e):
        return A @ s + e
    
    def B_ni_1(prop):
        random_value = np.random.uniform(0,1)

        if random_value < prop:
            return -1
        if random_value < 2*prop:
            return 1
        
        return 0

    A = generate_A()
    s = generate_s()
    e = generate_e()
    t = calculate_t(A, s, e)

    return A, s, t


A,s,t = key_gen()

print("s:", s)
print("t:", t)
print("A: ", A)

test:  [[16. 15.  0.  7.] [10. 12. 11.  6.]]

s: [[0.] [0.]]
t: [[16.  0.  0.  1.] [1. 0.]]
A:  [[[ 5.  2.  4. 14.] [ 5. 13.  8.  9.]]
 [[9. 2. 9. 6.] [12.  2.  7.  0.]]]


### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [10]:
from math import ceil

def encrypt(A, t, m):
    m = Znw(m)
    r = np.array(
        [Znw([-1, 1, 0, 0]), Znw([1, 1, 0, -1])]
    )
    e1 = np.array(
        [Znw([1, 1, 0]), Znw([1, 0, 0])]
    )
    e2 = Znw([-1, -1, 0, 0])

    u = A.T @ r + e1
    v = t.T @ r + e2 + ceil(Znw.q / 2) * m

    c = (u, v)
    return c

# Testowanie funkcji na podanych danych
A = np.array([
  [Znw([6, 16, 16, 11]), Znw([9, 4, 6, 3 ])],
  [Znw([5, 3,  10, 1 ]), Znw([6, 1, 9, 15])],
])
s = np.array(
  [Znw([-1, -1, 1, 0]), Znw([-1, 0, -1, 0])]
)
e = np.array(
  [Znw([1, 0, 0]), Znw([1, -1, 0])]
)

t = A @ s + e

m = [1, 0, 1, 1]

c = encrypt(A, t, m)
print(f"Szyfrogram c: {c}")

Szyfrogram c: (array([[11. 11. 10.  3.], [ 4.  4. 13. 11.]], dtype=object), [ 8.  6.  9. 16.])


### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [13]:
def decrypt(c,s):
    u, v = c
    mn = v - s.T @ u
    mn = mn.vec

    # Zaokrąglanie współczynników
    mn = [1 if 5 <= x < 13 else 0 for x in mn]
    
    return [0 for _ in range(n-len(mn))] + mn

mn = decrypt(c, s)
print(f"Odszyfrowana wiadomość mn: {mn}")

Odszyfrowana wiadomość mn: [1, 1, 0, 0]


### Testy

In [14]:
import secrets as sc

success = 0
for i in range(1000):
    output = []
    A,s,t = key_gen()
    
    m=[sc.choice((0,1)) for k in range(4)]
    
    c = encrypt(A,t,m)
    m_n = decrypt(c,s)

    if m != m_n:
        print(f'Error: {m} != {m_n}')

    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')


Error: [1, 1, 0, 1] != [1, 1, 0, 0]
Error: [1, 1, 1, 1] != [0, 1, 1, 1]
Error: [0, 0, 0, 0] != [0, 1, 0, 0]
Error: [1, 1, 0, 0] != [0, 1, 0, 0]
Error: [1, 0, 1, 1] != [1, 1, 1, 1]
Error: [1, 1, 0, 0] != [1, 0, 0, 0]
Error: [0, 1, 0, 1] != [1, 1, 0, 1]
Error: [1, 1, 0, 0] != [1, 0, 0, 0]
Error: [0, 0, 0, 1] != [0, 0, 0, 0]
Error: [0, 1, 0, 0] != [0, 0, 0, 0]
Error: [0, 0, 1, 1] != [0, 0, 0, 1]
Error: [0, 1, 0, 0] != [0, 0, 0, 0]
Error: [1, 0, 0, 0] != [1, 0, 1, 0]
Error: [1, 0, 1, 1] != [1, 0, 1, 0]
Error: [0, 1, 1, 1] != [0, 1, 0, 1]
Error: [1, 0, 0, 0] != [1, 1, 0, 0]
Success rate: 98.4 %
